In [49]:
import os
import openai
import backoff
import numpy as np
import pandas as pd


In [50]:
# Insert your key below. 
openai.api_key = None


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_response(model, prompt, temperature, max_tokens, top_p, frequency_penalty, presence_penalty):

    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)

    return response





In [51]:
def save_response(id,response):
    path = os.path.join("Dataset", str(id)+".txt") 
    with open(path, 'w') as f:
        f.write(response["choices"][0]["text"])

In [52]:
def generate_examples(number, start_id, prompt, model, essay_lengths_distribution, temperature_min, temperature_max, penalty_min =0, penalty_max = 0):
    ids = []
    temps = []
    freq_pens = []
    pres_pens = []
    essay_lengths = []

    

    for id in range(start_id, start_id+number):
        
        temperature = np.random.uniform(temperature_min, temperature_max)
        frequency_penalty = np.random.uniform(penalty_min, penalty_max)
        presence_penalty = np.random.uniform(penalty_min, penalty_max)

        # GPT-3 API expends max_tokens for the input prompt too, so we add the number of words in the input prompt to the desired length
        essay_length = np.random.choice(essay_lengths_distribution) + len(prompt.split())
        essay_length_tokens = int(essay_length * 1000 /750)

        resp = get_response(model=model, prompt=prompt, temperature=temperature, max_tokens=essay_length_tokens, top_p=1, frequency_penalty=frequency_penalty, presence_penalty=presence_penalty)
        save_response(id,resp)

        ids.append(id)
        temps.append(temperature)
        freq_pens.append(frequency_penalty)
        pres_pens.append(presence_penalty)
        essay_lengths.append(essay_length)

    

    df = pd.DataFrame({"IDs" : ids})

    df["Prompt"] = prompt
    df["Essay Length"] = essay_lengths
    df["Model"] = model
    df["Temperature"] = temps
    df["Frequency Penalty"] = freq_pens
    df["Presence Penalty"] = pres_pens

    df.to_csv("Parameters_for_IDS_" + str(start_id) + "_" + str(start_id+number-1)+".csv", index = False)


        

In [53]:
lengths_distribution = pd.read_csv("essay_lengths.csv")
lengths_distributions_1 = lengths_distribution["essay_length"][lengths_distribution.essay_set == 1]
lengths_distributions_2 = lengths_distribution["essay_length"][lengths_distribution.essay_set == 2]
lengths_distributions_7 = lengths_distribution["essay_length"][lengths_distribution.essay_set == 7]
lengths_distributions_8 = lengths_distribution["essay_length"][lengths_distribution.essay_set == 8]

In [54]:
essay_question = 'More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.'
generate_examples(number=500, start_id=0, prompt = essay_question, model = "text-davinci-003", essay_lengths_distribution=lengths_distributions_1, temperature_min=0, temperature_max=1, penalty_min =0, penalty_max = 1)

In [55]:
essay_question = 'Censorship in the Libraries. "All of us can think of a book that we hope none of our children or any other children have taken off the shelf. But if I have the right to remove that book from the shelf -- that work I abhor -- then you also have exactly the same right and so does everyone else. And then we have no books left on the shelf for any of us." --Katherine Paterson, Author. Write a persuasive essay to a newspaper reflecting your views on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.'
generate_examples(number=500, start_id=500, prompt = essay_question, model = "text-davinci-003", essay_lengths_distribution=lengths_distributions_2, temperature_min=0, temperature_max=1, penalty_min =0, penalty_max = 1)

In [56]:
essay_question = 'Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining. Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.'
generate_examples(number=500, start_id=1000, prompt = essay_question, model = "text-davinci-003", essay_lengths_distribution=lengths_distributions_7, temperature_min=0, temperature_max=1, penalty_min =0, penalty_max = 1)

In [57]:
essay_question = 'We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part.'
generate_examples(number=500, start_id=1500, prompt = essay_question, model = "text-davinci-003", essay_lengths_distribution=lengths_distributions_8, temperature_min=0, temperature_max=1, penalty_min =0, penalty_max = 1)